In [1]:
import operator
import random
import csv
import funcEval
import numpy as np
import GP_LS
import init_conf
import os.path
import time
import yaml
import copy
import smtplib
import socket
from deap import base
from deap import creator
from deap import tools
from deap import gp
import gp_conf as struct_gp
from shutil import copyfile
from conf_primitives import conf_sets, vector_benchmarks

In [2]:
#Evaluation Symbolic Regression (Best)
def evalSymbReg_b(individual, points, toolbox, config, type):
    points.flags['WRITEABLE'] = False
    func = toolbox.compile(expr=individual)
    if config["benchmark"]:
        vector = vector_benchmarks(config["problem"], points)
        data_x = copy.deepcopy(np.asarray(points)[:])
    else:
        vector = copy.deepcopy(points[config["num_var"]])
        data_x = copy.deepcopy(np.asarray(points)[:config["num_var"]])
    try:
        vector_x = func(*data_x)
    except TypeError:
        print (individual)
        print (data_x)
    with np.errstate(divide='ignore', invalid='ignore'):
        if isinstance(vector_x, np.ndarray):
            for e in range(len(vector_x)):
                if np.isnan(vector_x[e]) or np.isinf(vector_x[e]):
                    vector_x[e] = 0.
    if config["funcion"] == 1:
        d = './Results/%s/best_data_%d_%s.txt' % (config["problem"], config["n_problem"], type)
        GP_LS.ensure_dir(d)
        best_test = open(d, 'w')
        for item in vector_x:
            best_test.write("%s\n" % item)
        result = np.sum((vector_x - vector)**2)
        return np.sqrt(result/len(points[0])),
    elif config["funcion"] == 2:
        l = len(points[0])
        ME = np.sum(abs((vector_x - vector))) / l
        SD = np.sqrt(np.sum((vector_x - ME) ** 2) / (l))
        try:
            calc = 100 * (SD / ME)
            return calc,
        except ZeroDivisionError:
            return 100,

In [3]:
#Evaluation for Symbolic Regression (YAML)
def evalSymbReg(individual, points, toolbox, config):
    points.flags['WRITEABLE'] = False
    func = toolbox.compile(expr=individual)
    vector_x=[]
    if config["benchmark"]:
        vector = vector_benchmarks(config["problem"], points)
        data_x = copy.deepcopy(np.asarray(points)[:])
    else:
        vector = copy.deepcopy(points[config["num_var"]])
        data_x = copy.deepcopy(np.asarray(points)[:config["num_var"]])
    vector_x = func(*data_x)
    with np.errstate(divide='ignore', invalid='ignore'):
        if isinstance(vector_x, np.ndarray):
            for e in range(len(vector_x)):
                if np.isnan(vector_x[e]) or np.isinf(vector_x[e]):
                    vector_x[e] = 0.
    if config["funcion"] == 1:
        result = np.sum((vector_x - vector)**2)
        return np.sqrt(result/len(points[0])),
    elif config["funcion"] == 2:
        l = len(points[0])
        ME = np.sum(abs((vector_x - vector))) / l
        SD = np.sqrt(np.sum((vector_x - ME) ** 2) / (l))
        try:
            calc = 100 * (SD / ME)
            return calc,
        except ZeroDivisionError:
            return 100,

In [4]:
def train_test(n_corr, p, problem, name_database, toolbox, config, train_p, test_p):
    n_archivo = './data_corridas/%s/train_%d_%d.txt' % (problem, p, n_corr)
    #if config["test_flag"]:
    n_archivot = './data_corridas/%s/test_%d_%d.txt' % (problem, p, n_corr)
    if not (os.path.exists(n_archivo) or os.path.exists(n_archivot)):
        direccion="./data_corridas/%s/%s" %(problem, name_database)
        with open(direccion) as spambase:
            if config["db_name"][-3:]=="txt":
                spamReader = csv.reader(spambase,  delimiter=' ', skipinitialspace=True)
                num_c = sum(1 for line in open(direccion))
                num_r = len(next(csv.reader(open(direccion), delimiter=' ', skipinitialspace=True)))
            else:
                spamReader = csv.reader(spambase, delimiter=',', skipinitialspace=True)
                num_c = sum(1 for line in open(direccion))
                num_r = len(next(csv.reader(open(direccion), delimiter=',', skipinitialspace=True)))
            Matrix = np.empty((num_r, num_c,))
            for row, c in zip(spamReader, range(num_c)):
                for r in range(num_r):
                    try:
                        Matrix[r, c] = row[r]
                    except ValueError:
                        print ('Line {r} is corrupt', r)
                        break
        if not os.path.exists(n_archivo):
            long_train = int(len(Matrix.T) * train_p)
            data_train1 = random.sample(Matrix.T, long_train)
            np.savetxt(n_archivo, data_train1, delimiter=",", fmt="%s")
        if config["test_flag"]:
            if not os.path.exists(n_archivot):
                long_test = int(len(Matrix.T) * test_p)
                data_test1 = random.sample(Matrix.T, long_test)
                np.savetxt(n_archivot, data_test1, delimiter=",", fmt="%s")
    with open(n_archivo) as spambase:
        spamReader = csv.reader(spambase,  delimiter=',', skipinitialspace=True)
        num_c = sum(1 for line in open(n_archivo))
        num_r = len(next(csv.reader(open(n_archivo), delimiter=',', skipinitialspace=True)))
        Matrix = np.empty((num_r, num_c,))
        for row, c in zip(spamReader, range(num_c)):
            for r in range(num_r):
                try:
                    Matrix[r, c] = row[r]
                except ValueError:
                    print ('Line {r} is corrupt train' , r)
                    break
        data_train=Matrix[:]
    if config["test_flag"]:
        with open(n_archivot) as spambase:
            spamReader = csv.reader(spambase,  delimiter=',', skipinitialspace=True)
            num_c = sum(1 for line in open(n_archivot))
            num_r = len(next(csv.reader(open(n_archivot), delimiter=',', skipinitialspace=True)))
            Matrix = np.empty((num_r, num_c,))
            for row, c in zip(spamReader, range(num_c)):
                for r in range(num_r):
                    try:
                        Matrix[r, c] = row[r]
                    except ValueError:
                        print ('Line {r} is corrupt test' , r)
                        break
            data_test=Matrix[:]
    toolbox.register("evaluate", evalSymbReg, points=data_train, toolbox=toolbox, config=config)
    toolbox.register("evaluate_best_t", evalSymbReg_b, points=data_train, toolbox=toolbox, config=config, type="train")
    if config["test_flag"]:
        toolbox.register("evaluate_test", evalSymbReg, points=data_test, toolbox=toolbox, config=config)
        toolbox.register("evaluate_best", evalSymbReg_b, points=data_test, toolbox=toolbox, config=config, type="test")

In [5]:
def main(n_corr, p, problem, database_name, pset, config):


    pop_size = config["population_size"]
    cxpb                = config["cxpb"]  # 0.9
    mutpb               = config["mutpb"]  # 0.1
    train_p             = config["train_p"]
    test_p              = config["test_p"]
    tournament_size     = config["tournament_size"]
    ngen                = config["generations"]

    params              = ['best_of_each_specie', 2, 'yes']

    funcEval.LS_flag    = config["ls_flag"]
    LS_select           = config["ls_select"]
    funcEval.cont_evalp = 0
    num_salto           = config["num_salto"]  # 500
    cont_evalf          = config["cont_evalf"]

    SaveMatrix          = config["save_matrix"]
    GenMatrix           = config["gen_matrix"]
    version             = 3
    testing             = config["test_flag"]
    benchmark_flag      = config["benchmark"]


    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("FitnessTest", base.Fitness, weights=(-1.0,))
    creator.create("Individual", struct_gp.PrimitiveTree, fitness=creator.FitnessMin, fitness_test=creator.FitnessTest)

    toolbox = base.Toolbox()

    toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=0, max_=7)
    toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
    toolbox.register("population", init_conf.initRepeat, list, toolbox.individual)
    toolbox.register("compile", gp.compile, pset=pset)

    name_database=database_name
    direccion="./data_corridas/%s/train_%d_%d.txt"
    train_test(n_corr,p, problem, name_database, toolbox, config, train_p, test_p)

    toolbox.register("select", tools.selTournament, tournsize=tournament_size)
    toolbox.register("mate", struct_gp.cxSubtree)

    toolbox.register("expr_mut", gp.genHalfAndHalf, min_=0, max_=7)
    toolbox.register("mutate", struct_gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
    toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
    toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

    pop = toolbox.population(n=pop_size)
    hof = tools.HallOfFame(3)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    pop, log = GP_LS.GP_LS(pop, toolbox, cxpb, mutpb, ngen, funcEval.LS_flag, LS_select, cont_evalf,
                                   num_salto, SaveMatrix, GenMatrix, pset, n_corr, p,
                                   params, direccion, problem, testing, version, benchmark_flag, config,stats=mstats, halloffame=hof, verbose=True)

    return pop, log, hof

In [6]:
if __name__ == "__main__":

    config        = yaml.load(open("conf/conf.yaml"))
    problem       = config["problem"]
    num_var       = config["num_var"]
    database_name = config["db_name"]
    number        = config["n_problem"]

    c_f = './conf_record/%s/' % (problem)
    GP_LS.ensure_dir(c_f)
    copyfile('./conf/conf.yaml', ('./conf_record/%s/config_%s_%d.yaml'% (problem, problem, number)))

    d = './Timing/%s/timing_cxneat_%d.txt' % (problem, number)
    GP_LS.ensure_dir(d)
    time_conc = open(d, 'a')

    pset = conf_sets(num_var)

    instance_name=socket.gethostname()

    n = config["run_begin"]
    while n < config["run_end"]:
        begin_p = time.time()
        main(n, number, problem, database_name, pset, config)
        n += 1
        end_p = time.time()
        time_conc.write('\n%s;%s;%s;%s' % (n, begin_p, end_p, str(round(end_p - begin_p, 2))))


---- Generation 0 -----
Problem:  Housing2
Problem No.:  1
Run No.:  1
Local Search: True
Local Search Heuristic: 1 (LSHS)
Best Ind.: x10
Best Fitness: 11.129915913814784
Test fitness: 10.883850969235558
Avg Nodes: (7.64, 56, 1)
Evaluations:  100
---- Generation 1 -----
Problem:  Housing2
Problem No.:  1
Run No.:  1
Local Search: True
Local Search Heuristic: 1 (LSHS)
Fitness comp.:
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+

Num. LS: 60
Ind. Improvement: 60
Best Ind. LS: 1
Best Ind.: mysqrt(add(x5, x1))
Best Fitness: 6.133856549626232
Test fitness: 6.385001358375537
Avg Nodes: (7.18, 30, 1)
Evaluations:  1810
---- Generation 2 -----
Problem:  Housing2
Problem No.:  1
Run No.:  1
Local Search: True
Local Search Heuristic: 1 (LSHS)
Fitness comp.:
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+
+

Num. LS: 66
Ind. Improvement: 66
Be